# Hertziana IconClass Analytics

### What we have

1. Scanned images at the Hertziana Fotothek, each associated to at least one IconClass of Greek mytho9logy (94) or its characters (95)
2. The Keywords that IconClass associates to each class
3. The SenticNet ontology
4. The first work of UniL students, who extracted SenticNet information from the iconclass keywords [GitHub](https://github.com/unil-ish/Hertziana_IconClass)
5. The second work of UniL students, who annotated the actual images (and regions thereof) with core emotions from SenticNet

### Steps
1. Complete association of Fotothek Object IDs to the images loaded onto LabelStudio

## Setup

In [1]:
%pip install --quiet pandas rdflib scipy scikit-learn statsmodels krippendorff tabulate pingouin

Note: you may need to restart the kernel to use updated packages.


## Load Data

Two steps:
1. Load the output of the work done by the first batch of UniL students onto an RDF graph, which knows the emotions associated to each keyword of each IconClass of each relevant image in the Fototeca.
2. Load the dataset that associates each LabelStudio ID to one or more Fotothek Object IDs and, by extension, to one or more IconClasses

In [2]:
from rdflib import Graph, Namespace, URIRef
import re

# Prepare the environment
SNet = Namespace('http://example.org/SenticNet#')
XKOS = Namespace('http://rdf-vocabulary.ddialliance.org/xkos#')
src_imagedata = '../data/bhmpi/iconclass94_95_filtered_reconciled.csv'
src_senticrdf = '../data/bhmpi/fotothek_iconclasses_enriched.rdf'

# The 24 emotions taken from (SenticNet?)
emotions_index_en = ['acceptance','anger','annoyance','anxiety','bliss','calmness','contentment','delight',
                     'disgust','dislike','eagerness','ecstasy','enthusiasm','fear','grief','joy','loathing',
                     'melancholy','pleasantness','rage','responsiveness','sadness','serenity','terror']
# French labels were used by students - indices match those of English labels
emotions_index_fr = ['acceptation','colère','agacement','anxiété','félicité','calme','agrément','délice',
                     'dégoût','aversion','empressement','extase','enthousiasme','peur','deuil','joie','aversion',
                     'mélancolie','satisfaction','rage','réactivité','tristesse','sérénité','terreur']

In [3]:
def ic_clean(ic_dirty):
    # This regex should capture all the IconClasses of type 94 and 95
    preg_ic9x = re.compile(r"(9\d)\s*(\w+)(\s*(\d+|\([\w\s]+\)))?(\s*(\d+))?(\s*(\d+|\(\+\d\)))?")
    m = preg_ic9x.match(ic_dirty)
    if m:
        # Recreate the shorthand IconClass e.g. 94I561 or 95A(ACHILLES)1231
        # Skip odd capture groups after 1 as they are the ones that start with spaces
        ic_short = ''.join(filter(None, (m.group(1), m.group(2), m.group(4), m.group(6), m.group(8))))
        return ic_short
    raise ValueError('[ERROR] IconClass {} Not matched!'.format(ic))

1. Load the Fototeca Data from the previous project, where the students associated SenticNet emotions to each IconClass of relevant photos.

In [4]:
g = Graph()
g.parse(src_senticrdf)
len(g)

211622

2. Load the CSV data into a Pandas DataFrame, skipping the images that were not processed by UNIL students (i.e. they have a LabelStudio ID).

In [5]:
import csv
import pandas as pd

def parse_range_string(range_string):
    """Turns something like 1968-71 in a list [1968,1969,1970,1971]"""
    parts = range_string.split('-')
    common_prefix = parts[0][:-len(parts[-1])]
    start = int(common_prefix + parts[0][-len(parts[-1]):])
    end = int(common_prefix + parts[-1])
    return list(range(start, end + 1))

# Start with a list of lists, then turn it into a DataFrame.
iconclass_sents = []
rrange = re.compile(r"^(\d+)(\d{2})\-(\d{2})$")
with open(src_imagedata) as csvfile:
   reader = csv.reader(csvfile)
   next(reader, None) # Skip the CSV header, we'll rebuild it to our liking
   for r in reader:
       if r[0]:
           if rrange.match(r[0]): # Case of LabelStudio ID range string
               # Build one new row per item in range and repeat the data every time.
               # (TODO does this skew the results?)
               for lsid in parse_range_string(r[0]):
                   iconclass_sents.append([str(lsid), r[1].strip(), r[2].strip(), r[3].strip(), r[4].strip(), r[5].strip()])  
           else: # Case of single LabelStudio ID
               iconclass_sents.append([r[i].strip() for i in range(0,6)])

# Build the DataFrame
df = pd.DataFrame(iconclass_sents, columns = ['ID_LabelStudio','ID_Fotothek','IconClass_1','IconClass_2','IconClass_3','IconClass_4'])
df

,ID_LabelStudio,ID_Fotothek,IconClass_1,IconClass_2,IconClass_3,IconClass_4
0,71536480,8002844,94 C 13 3,,,
1,71536481,8003521,94 D 13 21 : 25 II 12,,,
2,71536482,8004274,94 C 11 3,,,
3,71536484,8004622,94 C 11 3,,,
4,71536485,8005026,94 C 11 3,,,
...,...,...,...,...,...,...
315,72971393,8156117,94 C 11 31,,,
316,72971394,8156258,94 C 13 3,,,
317,72971413,8156339,95 B (CIRCE),,,
318,72971415,8156642,94 C 22 (+0),,,


## Vectorize data from IconClass

We want to represent each image onto a sparse vector space where each position corresponds to one of the 24 emotions. The value in each cell reflects the number of times an emotions appears as associated to keywords of that image's IconClass.

#### Caveats
- [**FIXME**] An object may have multiple IconClasses: this may happen if e.g. multiple characters appear in a scene or the photo is part of a group for the same object (and single photos are not classified individually, or are they?).
- [**FIXME**] An item may be repeated on multiple rows with different IconClasses! (e.g. 71822038)

Load `senticnet`. This is a Python dictionary whose keys are concepts expressed in English, and whose values look like this:

```python
senticnet['concept_name'] = [
    'introspection_value', 
    'temper_value', 
    'attitude_value', 
    'sensitivity_value', 
    'primary_emotion', 
    'secondary_emotion', 
    'polarity_label', 
    'polarity_value', 
    'semantics1', 
    'semantics2', 
    'semantics3', 
    'semantics4', 
    'semantics5']
```

For the time being, we are interested in indices 4 and 5, i.e. `primary_emotion` and `secondary_emotion`.

First define the function.

In [6]:
from rdflib.namespace import DC, SKOS
from senticnet.senticnet import senticnet
import urllib.parse

#patch SenticNet
senticnet['calmness'] = [0, 0.66, 0, 0, '#calmness', None ]
senticnet['eagerness'] = [0, 0, 0, 0.66, '#eagerness', None ]
senticnet['pleasantness'] = [0, 0, 0.66, 0, '#pleasantness', None ]
senticnet['sadness'] = [-0.66, 0, 0, 0, '#sadness', None ]

# Sparse pseudo-matrix, indexed by Fotothek ID
fotothek_vectors = {}
fotothek_dimensions = {} # This will collect values in the (introspection, temper, attitude, sensitivity) range
# Keep track of IconClasses with no SenticNet match
unmatched = set()

def fotothek_ic(foto_id, graph=g, target=fotothek_vectors, target_dim=fotothek_dimensions):
    """
    Processes data of one image and populates the pseudo-matrix with the resulting sparse vector.
    
    FIXME don't take the image's IconClasses from the RDF graph: take them from the DataFrame
    TODO This can be made faster, but for 300 rows we can live with it
    """

    # This regex should capture all the IconClasses of type 94 and 95
    preg_ic9x = re.compile(r"(9\d)\s*(\w+)(\s*(\d+|\([\w\s]+\)))?(\s*(\d+))?(\s*(\d+|\(\+\d\)))?")

    # Get the rows with the same Fotothek ID (there may be multiple) and process the together.
    dfr = df.loc[df['ID_Fotothek'] == foto_id[1:]]
    for i, rr in dfr.iterrows():
        for ic in (rr['IconClass_1'], rr['IconClass_2'], rr['IconClass_3'], rr['IconClass_4']):
            m = preg_ic9x.match(ic)
            if ic and not m:
                print('[ERROR] IconClass {} Not matched!'.format(ic))
            if m:
                # Recreate the shorthand IconClass e.g. 94I561 or 95A(ACHILLES)1231
                # Skip odd capture groups after 1 as they are the ones that start with spaces
                ic_short = ''.join(filter(None, (m.group(1), m.group(2), m.group(4), m.group(6), m.group(8))))

                # Now query the graph for associated emotions using in-memory SPARQL
                x_ic = URIRef('http://iconclass.org/{}'.format(urllib.parse.quote(ic_short)))
                res = graph.query(f"""
SELECT * WHERE {{
  <{x_ic}> <{SNet.hasSenticNet}>/<{DC.subject}> ?sub .
}}
""")
                if len(res) == 0:
                    unmatched.add(ic)
                # Reuse an existing vector for this image, if any, or initialise to all zeroes
                vec = target[foto_id] if foto_id in target else ( [0] * len(emotions_index_en) )
                vec2 = target_dim[foto_id] if foto_id in target_dim else ( [0.0] * 4 )
                for r in res:
                    i = str(r[0]) # the word that we will look up in senticnet
                    if i in senticnet:
                        # Increase the counter for that image and emotion on every match
                        # TODO better to use TF/IDF ?
                        if senticnet[i][4]: # primary emotion
                            sent_1 = senticnet[i][4]
                            idx = emotions_index_en.index(sent_1[1:])
                            vec[idx] += 1
                        if senticnet[i][5]: # secondary emotion
                            sent_1 = senticnet[i][5]
                            idx = emotions_index_en.index(sent_1[1:])
                            vec[idx] += 1
                        # introspection, temper, attitude, sensitivity
                        for j, v in enumerate(vec2):
                            vec2[j] = v + senticnet[i][j]                      
                target[foto_id] = vec
                target_dim[foto_id] = vec2

Now execute it.

In [7]:
# Populate the matrix
for index, row in df.iterrows():
    idf = f"0{row['ID_Fotothek']}"
    fotothek_ic(idf, g)
fotothek_dimensions

{'08002844': [0.193, 0.306, 0.029, 0.0],
 '08003521': [0.08799999999999997, 0.0, -1.335, -1.1360000000000001],
 '08004274': [0.899, 0.896, 0.807, 0.741],
 '08004622': [0.899, 0.896, 0.807, 0.741],
 '08005026': [0.899, 0.896, 0.807, 0.741],
 '08005109': [0.951, 0.0, 0.721, 0.0],
 '08005519': [0.0, 0.0, 0.0, 0.0],
 '08006579': [0.994, 0.0, 1.512, 0.874],
 '08006588': [1.859, 0.39, -0.7849999999999999, -0.895],
 '08006590': [0.0, 0.0, -0.249, 0.0],
 '08006591': [1.186, 0.909, 0.0, 0.0],
 '08006607': [0.0, 0.0, 0.0, 0.0],
 '08007401': [-3.2, 0.612, -1.484, 0.0],
 '08008229': [1.071, 0.0, 6.839999999999999, 0.0],
 '08009086': [0.35199999999999987, 0.0, -5.34, -4.5440000000000005],
 '08009087': [7.355999999999999, 0.0, 3.072, 2.944],
 '08009088': [3.54, 0.0, 3.08, 0.0],
 '08010428': [0.0, 0.0, 0.0, 0.0],
 '08011187': [0.0, 0.0, 0.0, 0.0],
 '08011716': [1.885, 0.0, 1.545, 0.0],
 '08012461': [0.899, 0.896, 0.807, 0.741],
 '08013337': [0.0, 0.0, 0.0, 0.0],
 '08013396': [0.962, 0.0, 0.784, 0.0],

Print the IconClasses for which we didn't find a match in SenticNet:

In [8]:
unmatched

{'94 C 1',
 '94 C 11 31',
 '94 C 14',
 '94 D 22',
 '94 F 31',
 '94 F 32 1',
 '94 F 42 21',
 '94 F 74 2',
 '94 G',
 '94 H 15 1',
 '94 I 21',
 '94 I 23',
 '94 I 23 4',
 '94 I 52',
 '94 I 52 & 92 B 5',
 '94H231',
 '94I52',
 '94I561',
 '95 A ( ) 7',
 '95 A (ACHILLES) 1',
 '95 A (ACHILLES) 12 31',
 '95 A (ACHILLES) 12 31 : 48 C 73 21',
 '95 A (ACHILLES) 21',
 '95 A (ADMETUS)',
 '95 A (ANCHISES)',
 '95 A (CEPHALUS) 21',
 '95 A (DAEDALUS)',
 '95 A (GERYON)',
 '95 A (HECTOR)',
 '95 A (HIPPOLYTUS)',
 '95 A (ICARUS)',
 '95 A (LAOMEDON) 31',
 '95 A (LAOMEDON) 32',
 '95 A (ORESTES)',
 '95 A (PARIS)',
 '95 A (PARIS) 12 12',
 '95 A (PHILEMON AND BAUCIS)',
 '95 A (PHOENIX)',
 '95 A (PYRAMUS AND THISBE)',
 '95 A (TANTALUS)',
 '95 A (TELEMACHUS)',
 '95 A (TELEMACHUS) 31',
 '95 A (ULYSSES)',
 '95 A (ULYSSES) 2',
 '95 B',
 '95 B ( ) 7',
 '95 B (AMAZONS) 4',
 '95 B (ANDROMACHE) 6',
 '95 B (ANTIOPE)',
 '95 B (ARIADNE) 61',
 '95 B (DANAE)',
 '95 B (DANAE) 2',
 '95 B (GALATHEA)',
 '95 B (HELEN)',
 '95 B (IOL

In [9]:
# Show some critical examples
for key, value in fotothek_vectors.items():
    if key in ('08006588','08008229','08046444','08048360','08049596'):
        print(f"{key}: {value}")

08006588: [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
08008229: [0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 9, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0]
08046444: [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0]
08048360: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0]
08049596: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
# Show some critical examples
for key, value in fotothek_dimensions.items():
    if key in ('08006588','08008229','08046444','08048360','08049596'):
        print(f"{key}: {value}")

08006588: [1.859, 0.39, -0.7849999999999999, -0.895]
08008229: [1.071, 0.0, 6.839999999999999, 0.0]
08046444: [-4.476, -1.98, 0.0, 2.511]
08048360: [-13.312000000000005, 0.0, 0.0, 0.0]
08049596: [0.0, 0.0, 0.0, 0.0]


In [11]:
# Calculate zero indices
num_lists = len(fotothek_vectors)

# Length of the lists (assuming all lists are of the same length)
list_length = len(next(iter(fotothek_vectors.values())))

# Initialize a list to hold the indices where the value is zero for all lists
zero_indices = []

# Iterate through each index
for i in range(list_length):
    # Check if all lists have a zero at this index
    if all(lst[i] == 0 for lst in fotothek_vectors.values()):
        zero_indices.append(i)

print("Indices where the value is zero for all elements: {}".format([emotions_index_en[j] for j in zero_indices]))

Indices where the value is zero for all elements: ['annoyance', 'responsiveness']


In [12]:
for key, value in fotothek_vectors.items():
    i = 19
    if value[i] != 0:
        print('[INFO] Found a {} value of {} for item {}'.format(emotions_index_en[i], value[i], key))

[INFO] Found a rage value of 1 for item 08023383
[INFO] Found a rage value of 1 for item 08025758
[INFO] Found a rage value of 4 for item 08147770


#### Use TF-IDF

Absolute occurrences are dangerous. let's try to represent each image as a document, e.g.

`08006588 = "calmness delight disgust dislike ecstasy ecstasy loathing terror"`

In [13]:
corpus = []
for key, value in fotothek_vectors.items():
    corpus.append(" ".join(' '.join(t * [emotions_index_en[i]]) for i,t in enumerate(value)))
len(corpus)

268

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

datt = []
for key, value in fotothek_vectors.items():
    datt.append(value)

transformer = TfidfTransformer()
transformed_data = transformer.fit_transform(datt)
print(transformed_data)

  (0, 0)	0.5384006269438193
  (0, 6)	0.6143519049575892
  (0, 22)	0.5767984932205387
  (1, 3)	0.49294858042442236
  (1, 11)	0.2570334859173578
  (1, 14)	0.2727909668409128
  (1, 16)	0.6404120711380742
  (1, 23)	0.4543049105230541
  (2, 4)	0.639926463773375
  (2, 7)	0.40522756268075893
  (2, 11)	0.37470201455789026
  (2, 12)	0.5346804126696371
  (3, 4)	0.639926463773375
  (3, 7)	0.40522756268075893
  (3, 11)	0.37470201455789026
  (3, 12)	0.5346804126696371
  (4, 4)	0.639926463773375
  (4, 7)	0.40522756268075893
  (4, 11)	0.37470201455789026
  (4, 12)	0.5346804126696371
  (5, 7)	0.7342199865454935
  (5, 11)	0.678911637370531
  (7, 7)	0.7787359670408394
  (7, 11)	0.3600371279393532
  (7, 12)	0.5137543772486521
  :	:
  (254, 15)	0.535115052797589
  (254, 16)	0.33041811064187926
  (254, 18)	0.3909572346464148
  (255, 1)	0.4220212598023994
  (255, 11)	0.19664670278730928
  (255, 12)	0.2806046834856227
  (255, 14)	0.2087021618524651
  (255, 16)	0.7349329704942708
  (255, 23)	0.347571688550986

### Vectorize student annotations

We want to represent each annotated image in the same way as before (vector space as long as are the emotions), but this time, the values are taken from the annotations of the second batch of users.

#### Caveats
- [**FIXME**] Multiple LabelStudio IDs may correspond to one object (if e.g. they were photos of the same object from different angles, or colored vs. b/w). Example: `75990783-85` is a range for object ID `08008229`.
- [**FIXME**] Something is causing anomalous prints (check the output).

In [15]:
import json
with open('../data/unil/senticlassics_anon.json') as f:
    annotations = json.load(f)

In [16]:
# Set column 'B' as the index
df.set_index('ID_LabelStudio', inplace=True)

In [17]:
ls_vectors = {}
ls_dimensions = {}
for task in annotations:
    vec = [0] * len(emotions_index_fr)
    vec2 = [0] * 4 # introspection, temper, attitude, sensitivity
    id_ls = task['id']
    for ann in task['annotations']:
        for annr in ann['result']:
            if annr['type']=='polygonlabels':
                if len(annr['value']['polygonlabels']) > 1:
                    print("[WARN] annotation array longer than 1")
                sent = annr['value']['polygonlabels'][0]
                try:
                    idx = emotions_index_fr.index(sent)
                    vec[idx] += 1
                    for j, v in enumerate(vec2):
                        vec2[j] = v + senticnet[emotions_index_en[idx]][j]
                except ValueError:
                    pass
    try:
        locc = df.loc[str(id_ls)]
        idx = locc['ID_Fotothek'].unique()[0] if isinstance(locc, pd.DataFrame) else locc['ID_Fotothek']
        foto_id = f"0{idx}"
        if foto_id in ls_vectors:
            print('[WARN] Replacing pre-computed photo with ID {}. These should be merged in the future.'.format(foto_id))
        ls_vectors[foto_id] = vec
        ls_dimensions[foto_id] = vec2
    except KeyError:
        print('[WARN] No match for LabelStudio ID {} in IconClass table - part of a range?'.format(id_ls))

for key, value in ls_vectors.items():
    print(f"{key}: {value}")
for key, value in ls_dimensions.items():
    print(f"{key}: {value}")

[WARN] Replacing pre-computed photo with ID 08025602. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08025910. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08048360. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08048360. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08048360. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08049596. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08050623. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08055057. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08055057. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08056895. These should be merged in the future.
[WARN] Replacing pre-computed photo with ID 08065347. These should be merged in 

In [18]:
datt2 = []
for key, value in ls_vectors.items():
    datt2.append(value)

transformer = TfidfTransformer()
transformed_data2 = transformer.fit_transform(datt2)
print(transformed_data2)

  (0, 1)	0.3156115698767177
  (0, 2)	0.16909130673603023
  (0, 3)	0.29842550888142066
  (0, 9)	0.16909130673603023
  (0, 10)	0.15856333355714966
  (0, 13)	0.41512802162410606
  (0, 19)	0.4331713594396486
  (0, 20)	0.44358776174913317
  (0, 21)	0.16406728610376242
  (0, 23)	0.38124612012880765
  (1, 0)	0.076326315553647
  (1, 1)	0.2877356484051995
  (1, 2)	0.061662627643693686
  (1, 3)	0.05441350412609291
  (1, 5)	0.19840686212359313
  (1, 6)	0.07736119592118762
  (1, 8)	0.5910602805753409
  (1, 9)	0.30831313821846845
  (1, 10)	0.2891169269376144
  (1, 11)	0.09457591607311722
  (1, 13)	0.15138498315487933
  (1, 14)	0.07736119592118762
  (1, 18)	0.43774695329422075
  (1, 19)	0.07898242895697055
  (1, 20)	0.26960568925774675
  :	:
  (264, 1)	0.53545011300654
  (264, 3)	0.25314656955859993
  (264, 9)	0.28687148362652576
  (264, 10)	0.2690102739422186
  (264, 15)	0.3311847545374454
  (264, 17)	0.3394198472753199
  (264, 20)	0.25085594638759046
  (264, 22)	0.2839731063386777
  (265, 0)	0.391

## Assess reliability of agreement

Fleiss' Kappa cannot be applied, because it assumes the same number of ratings, wheras our annotators have each entered an arbitrary amount per image.

One thing we can do is compute Krippendorff's Alpha for both matrices and compare the values.

In [19]:
import numpy as np
import krippendorff
from tabulate import tabulate

def kripp(data):
    reliability_data = [[np.nan if v == 0 else int(v) for v in rating] for fotothek, rating in data.items()]
    a_nom = krippendorff.alpha(reliability_data=reliability_data, level_of_measurement="nominal")
    a_int = krippendorff.alpha(reliability_data=reliability_data)
    return a_nom, a_int

a_nom, a_int = kripp(fotothek_vectors)
a_nom2, a_int2 = kripp(ls_vectors)
print("Here are Krippendorff's Alphas:")
print()
print(tabulate([['nominal', a_nom, a_nom2], ['interval', a_int, a_int2]], headers=['Metric', 'IconClass', 'Annotations']))

Here are Krippendorff's Alphas:

Metric      IconClass    Annotations
--------  -----------  -------------
nominal    0.0210043       0.0133541
interval  -0.00706485      0.0356592


Another approach is to consider only two raters per item: one is the IconClass system, the others are the students as a whole.

Then we can decide that each item is a Fotothek object, in which case we need 24 measures.

For example, let's take index 1 (anger)

In [20]:
ratings_0 =  ([ v[1] for i, v in fotothek_vectors.items() ], [ v[1] for i, v in ls_vectors.items() ])
ratings_1 =  ([ v[1] for i, v in fotothek_vectors.items() ], [ v[1] for i, v in ls_vectors.items() ])

print(len(ratings_0[1]))
a_nom = krippendorff.alpha(reliability_data=ratings_1, level_of_measurement="nominal")
a_int = krippendorff.alpha(reliability_data=ratings_1)

print("Here are Krippendorff's Alphas:")
print()
print(tabulate([['nominal', a_nom ], ['interval', a_int ]], headers=['Metric', 'Anger']))

268
Here are Krippendorff's Alphas:

Metric         Anger
--------  ----------
nominal   -0.115232
interval  -0.0532849


In [21]:
def alpha_emotion(e_index, rater_1, rater_2, zeroes_matter=True):
    if zeroes_matter:
        ratings =  ([ v[e_index] for i, v in rater_1.items() ], 
                    [ v[e_index] for i, v in rater_2.items() ])
    else:
        ratings =  ([ np.nan if v[e_index] == 0 else int(v[e_index]) for i, v in rater_1.items() ], 
                    [ np.nan if v[e_index] == 0 else int(v[e_index]) for i, v in rater_2.items() ])
    try:
        a_nom = krippendorff.alpha(reliability_data=ratings, level_of_measurement="nominal")
    except ValueError:
         a_nom = np.nan
    return a_nom

table = []

for e, emo in enumerate(emotions_index_en):
    a_nom = alpha_emotion(e, fotothek_vectors, ls_vectors, True)
    a_nom2 = alpha_emotion(e, fotothek_vectors, ls_vectors, False)
    table.append([emo, a_nom,  a_nom2])

print("Rater agreement measure (Krippendorff's Alpha)")
print("----------------------------------------------------")
print(tabulate(table, headers=['Emotion', 'Nominal-Zeroes', 'Nominal-Nozeroes' ]))

Rater agreement measure (Krippendorff's Alpha)
----------------------------------------------------
Emotion           Nominal-Zeroes    Nominal-Nozeroes
--------------  ----------------  ------------------
acceptance           -0.0193617           -0.125
anger                -0.115232             1
annoyance            -0.134902           nan
anxiety              -0.145026            -0.166667
bliss                 0.015383           nan
calmness             -0.199496             0
contentment          -0.0678976          nan
delight              -0.00684346          -0.222222
disgust              -0.0108114            0.0909091
dislike              -0.122693            -0.241379
eagerness            -0.148272             0
ecstasy              -0.0330886           -0.0714286
enthusiasm           -0.015207           nan
fear                 -0.194793             0
grief                 0.0335808           -0.08
joy                  -0.0757661          nan
loathing             -0.058738

/Users/adamou/.pyenv/versions/3.9.18/envs/emoclassics/lib/python3.9/site-packages/krippendorff/krippendorff.py:364: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - (o * d).sum() / (e * d).sum()


#### Using TF-IDF

In [22]:
def alpha_emotion_tfidf(e_index, zeroes_matter=True):
    tf_f_d = transformed_data.todense()
    tf_l_d = transformed_data2.todense()
    if zeroes_matter:
        ratings =  ([ tf_f_d[i, e_index] for i in range(tf_f_d.shape[0]) ], 
                    [ tf_l_d[i, e_index] for i in range(tf_l_d.shape[0])  ])
    else:
        ratings =  ([ np.nan if tf_f_d[i, e_index] == 0 else tf_f_d[i, e_index] for i in range(tf_f_d.shape[0]) ], 
                    [ np.nan if tf_l_d[i, e_index] == 0 else tf_l_d[i, e_index] for i in range(tf_l_d.shape[0])  ])
    try:
        a_nom = krippendorff.alpha(reliability_data=ratings, level_of_measurement="nominal")
    except ValueError:
         a_nom = np.nan
    return a_nom

table = []

for e, emo in enumerate(emotions_index_en):
    a_nom = alpha_emotion_tfidf(e, True)
    a_nom2 = alpha_emotion_tfidf(e, False)
    table.append([emo, a_nom,  a_nom2])

print("Rater agreement measure (Krippendorff's Alpha)")
print("----------------------------------------------------")
print(tabulate(table, headers=['Emotion', 'Nominal-Zeroes', 'Nominal-Nozeroes' ]))

Rater agreement measure (Krippendorff's Alpha)
----------------------------------------------------
Emotion           Nominal-Zeroes    Nominal-Nozeroes
--------------  ----------------  ------------------
acceptance           -0.00334896          -0.153846
anger                -0.0998417            0
annoyance            -0.0974359          nan
anxiety              -0.103666            -0.037037
bliss                -0.00540287           0
calmness             -0.147556             0
contentment          -0.0490196            0
delight              -0.0185868            0
disgust              -0.00235046          -0.0714286
dislike              -0.067884            -0.0227273
eagerness            -0.103322             0
ecstasy              -0.0376579           -0.025641
enthusiasm           -0.0272865           -0.037037
fear                 -0.146263             0
grief                -0.00555318          -0.0243902
joy                  -0.0626454            0
loathing             -

## IconClass-based Evaluation

Another approach that we can try is to measure the rater agreement over IconClass categories

In [23]:
# Sparse pseudo-matrix, indexed by Fotothek ID
fotothek_ic_vectors = {}
fotothek_ic_dimensions = {}
# Keep track of IconClasses with no SenticNet match
unmatched_ic = set()

def fotothek_ic_by_ic(foto_id, graph=g, target=fotothek_ic_vectors, target_dim=fotothek_ic_dimensions):
    """
    Processes data of one image and populates the pseudo-matrix with the resulting sparse vector.
    
    FIXME don't take the image's IconClasses from the RDF graph: take them from the DataFrame
    TODO This can be made faster, but for 300 rows we can live with it
    """

    # This regex should capture all the IconClasses of type 94 and 95
    preg_ic9x = re.compile(r"(9\d)\s*(\w+)(\s*(\d+|\([\w\s]+\)))?(\s*(\d+))?(\s*(\d+|\(\+\d\)))?")

    # Get the rows with the same Fotothek ID (there may be multiple) and process the together.
    dfr = df.loc[df['ID_Fotothek'] == foto_id[1:]]
    for i, rr in dfr.iterrows():
        for ic in (rr['IconClass_1'], rr['IconClass_2'], rr['IconClass_3'], rr['IconClass_4']):
            m = preg_ic9x.match(ic)
            if ic and not m:
                print('[ERROR] IconClass {} Not matched!'.format(ic))
            if m:
                # Recreate the shorthand IconClass e.g. 94I561 or 95A(ACHILLES)1231
                # Skip odd capture groups after 1 as they are the ones that start with spaces
                ic_short = ''.join(filter(None, (m.group(1), m.group(2), m.group(4), m.group(6), m.group(8))))

                # Now query the graph for associated emotions using in-memory SPARQL
                x_ic = URIRef('http://iconclass.org/{}'.format(urllib.parse.quote(ic_short)))
                res = graph.query(f"""
SELECT * WHERE {{
  <{x_ic}> <{SNet.hasSenticNet}>/<{DC.subject}> ?sub .
}}
""")
                if len(res) == 0:
                    unmatched_ic.add(ic_short)
                # Reuse an existing vector for this image, if any, or initialise to all zeroes
                vec = target[ic_short] if ic_short in target else ( [0] * len(emotions_index_en) )
                vec2 = target_dim[ic_short] if ic_short in target_dim else ( [0.0] * 4 )
                for r in res:
                    i = str(r[0]) # the word that we will look up in senticnet
                    if i in senticnet:
                        # Increase the counter for that image and emotion on every match
                        # TODO better to use TF/IDF ?
                        if senticnet[i][4]: # primary emotion
                            sent_1 = senticnet[i][4]
                            idx = emotions_index_en.index(sent_1[1:])
                            vec[idx] += 1
                        if senticnet[i][5]: # secondary emotion
                            sent_1 = senticnet[i][5]
                            idx = emotions_index_en.index(sent_1[1:])
                            vec[idx] += 1
                        # introspection, temper, attitude, sensitivity
                        for j, v in enumerate(vec2):
                            vec2[j] = v + senticnet[i][j] 
                target[ic_short] = vec
                target_dim[ic_short] = vec2

In [24]:
# Populate the matrix
for index, row in df.iterrows():
    idf = f"0{row['ID_Fotothek']}"
    fotothek_ic_by_ic(idf, g)

ic_list = list(fotothek_ic_vectors.keys())
ic_list.sort()

# Show some critical examples
for key, value in fotothek_ic_vectors.items():
    #if key in ('08006588','08008229','08046444','08048360','08049596'):
    print(f"{key}: {value}")

94C133: [11, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0]
94D1321: [0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 7, 0, 14, 0, 0, 0, 0, 0, 0, 7]
94C113: [0, 0, 0, 0, 12, 0, 0, 12, 0, 0, 0, 12, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94F54: [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
95B(AMAZONS)4: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94I161: [0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
95A(AGAMEMNON)4: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94F112: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
94F12: [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94F8311: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94F73: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94F61: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94F31:

In [25]:
ls_ic_vectors = {}
ls_ic_dimensions = {}
for task in annotations:
    vec = [0] * len(emotions_index_fr)
    vec2 = [0] * 4
    id_ls = task['id']
    for ann in task['annotations']:
        for annr in ann['result']:
            if annr['type']=='polygonlabels':
                if len(annr['value']['polygonlabels']) > 1:
                    print("[WARN] annotation array longer than 1")
                sent = annr['value']['polygonlabels'][0]
                try:
                    idx = emotions_index_fr.index(sent)
                    vec[idx] += 1
                    for j, v in enumerate(vec2):
                        vec2[j] = v + senticnet[emotions_index_en[idx]][j]
                except ValueError:
                    pass

    locc = df.loc[str(id_ls)]
    for ic in (locc['IconClass_1'], locc['IconClass_2'], locc['IconClass_3'], locc['IconClass_4']):
        if isinstance(ic, pd.Series):
            indices = [ x[1] for x in ic.items()]
        elif isinstance(ic, str):
            indices = [ ic ]
        else:
            indices = []
        for i in indices:
            if i and i!='':
                icc = ic_clean(i)
                if icc in ls_ic_vectors:
                    ls_ic_vectors[icc] = [x + y for x, y in zip(ls_ic_vectors[icc], vec)]
                else:
                    ls_ic_vectors[icc] = vec
                if icc in ls_ic_dimensions:
                    ls_ic_dimensions[icc] = [x + y for x, y in zip(ls_ic_dimensions[icc], vec2)]
                else:
                    ls_ic_dimensions[icc] = vec2


for key, value in ls_ic_vectors.items():
    print(f"{key}: {value}")
for key, value in ls_ic_dimensions.items():
    print(f"{key}: {value}")

94C133: [2, 8, 2, 8, 0, 1, 0, 0, 1, 3, 8, 1, 3, 11, 1, 1, 0, 2, 0, 3, 10, 4, 2, 7]
94D1321: [1, 15, 2, 7, 0, 8, 1, 0, 10, 6, 11, 1, 1, 13, 1, 0, 0, 1, 6, 3, 9, 6, 2, 6]
94C113: [9, 8, 10, 11, 3, 31, 10, 0, 3, 4, 12, 4, 7, 10, 2, 27, 0, 2, 8, 3, 25, 4, 12, 2]
94F54: [0, 1, 2, 4, 0, 4, 2, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 2, 0, 3, 8, 1, 1, 1]
95B(AMAZONS)4: [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0]
94I161: [2, 11, 14, 4, 0, 11, 0, 1, 1, 4, 4, 1, 1, 10, 0, 0, 0, 1, 6, 4, 1, 5, 5, 4]
95A(AGAMEMNON)4: [0, 23, 9, 3, 0, 3, 3, 0, 2, 4, 6, 0, 0, 42, 3, 0, 0, 4, 6, 11, 16, 19, 1, 6]
94F112: [0, 23, 9, 3, 0, 3, 3, 0, 2, 4, 6, 0, 0, 42, 3, 0, 0, 4, 6, 11, 16, 19, 1, 6]
94F12: [0, 25, 9, 6, 0, 3, 3, 1, 3, 7, 7, 0, 1, 50, 5, 0, 0, 4, 8, 12, 20, 24, 1, 10]
94F8311: [0, 23, 9, 3, 0, 3, 3, 0, 2, 4, 6, 0, 0, 42, 3, 0, 0, 4, 6, 11, 16, 19, 1, 6]
94F73: [0, 15, 5, 7, 0, 2, 1, 0, 2, 5, 12, 0, 0, 17, 3, 1, 0, 1, 0, 50, 25, 11, 2, 15]
94F61: [0, 15, 5, 7, 0, 2, 1, 0, 2, 5, 12, 0, 0

In [26]:
def alpha_iconclass(e_index, rater_1, rater_2, zeroes_matter=True):
    if zeroes_matter:
        ratings =  ( rater_1[e_index], rater_2[e_index] )
    else:
        ratings =  ([ np.nan if v == 0 else int(v) for v in rater_1[e_index] ], 
                    [ np.nan if v == 0 else int(v) for v in rater_2[e_index]])
    try:
        a_nom = krippendorff.alpha(reliability_data=ratings, level_of_measurement="nominal")
    except ValueError:
         a_nom = np.nan
    return a_nom

table = []

for e in ic_list:
    a_nom = alpha_iconclass(e, fotothek_ic_vectors, ls_ic_vectors, True)
    a_nom2 = alpha_iconclass(e, fotothek_ic_vectors, ls_ic_vectors, False)
    table.append([e, a_nom ])

print("Rater agreement measure (Krippendorff's Alpha)")
print("----------------------------------------------------")
print(tabulate(table, headers=['IconClass', 'Nominal-Zeroes' ]))

Rater agreement measure (Krippendorff's Alpha)
----------------------------------------------------
IconClass                   Nominal-Zeroes
------------------------  ----------------
94C                            -0.179211
94C1                           -0.24649
94C111                         -0.104027
94C112                         -0.0681818
94C113                         -0.243959
94C1131                        -0.321516
94C131                         -0.124402
94C133                         -0.212903
94C14                          -0.119048
94C21                           0.00704225
94C22                          -0.228223
94C22(+0)                      -0.228758
94D132                         -0.289976
94D1321                        -0.115065
94D22                           0
94F                            -0.130584
94F112                         -0.19323
94F113                         -0.122867
94F12                          -0.222367
94F121                         -0.0966667

In [27]:
len(ls_ic_dimensions)

173

In [28]:
len(fotothek_ic_dimensions)

173

In [29]:
ratings = []
for i,it in ls_ic_dimensions.items():
    ratings.append(
        { 'ID' : i, 'Rater' : "students", "introspection": it[0], "temper": it[1], "attitude": it[2], "sensitivity": it[3] }
    )
for i,it in fotothek_ic_dimensions.items():
    ratings.append(
        { 'ID' : i, 'Rater' : "Fotothek", "introspection": it[0], "temper": it[1], "attitude": it[2], "sensitivity": it[3] }
    )
ratings

[{'ID': '94C133',
  'Rater': 'students',
  'introspection': -2.6400000000000006,
  'temper': -7.620000000000001,
  'attitude': -0.99,
  'sensitivity': -5.3199999999999985},
 {'ID': '94D1321',
  'Rater': 'students',
  'introspection': -3.96,
  'temper': -7.62,
  'attitude': -4.290000000000001,
  'sensitivity': -5.659999999999999},
 {'ID': '94C113',
  'Rater': 'students',
  'introspection': 19.819999999999997,
  'temper': 15.840000000000002,
  'attitude': 4.950000000000001,
  'sensitivity': 10.940000000000001},
 {'ID': '94F54',
  'Rater': 'students',
  'introspection': -0.66,
  'temper': -1.3499999999999999,
  'attitude': 0,
  'sensitivity': 0.9800000000000002},
 {'ID': '95B(AMAZONS)4',
  'Rater': 'students',
  'introspection': 1,
  'temper': 0.66,
  'attitude': 0.66,
  'sensitivity': 1.33},
 {'ID': '94I161',
  'Rater': 'students',
  'introspection': -2.6300000000000003,
  'temper': -6.97,
  'attitude': 3.64,
  'sensitivity': -7.950000000000001},
 {'ID': '95A(AGAMEMNON)4',
  'Rater': 'st

In [30]:
import pingouin as pg
df = pd.DataFrame(ratings)
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='introspection').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,-0.132,0.767,172,173,0.959,"[-0.28, 0.02]"
1,ICC2,Single random raters,-0.122,0.780,172,172,0.948,"[-0.26, 0.03]"
2,ICC3,Single fixed raters,-0.124,0.780,172,172,0.948,"[-0.27, 0.03]"
3,ICC1k,Average raters absolute,-0.304,0.767,172,173,0.959,"[-0.76, 0.03]"
4,ICC2k,Average random raters,-0.277,0.780,172,172,0.948,"[-0.71, 0.05]"
5,ICC3k,Average fixed raters,-0.283,0.780,172,172,0.948,"[-0.73, 0.05]"


In [31]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='temper').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,-0.006,0.988,172,173,0.531,"[-0.15, 0.14]"
1,ICC2,Single random raters,0.010,1.022,172,172,0.445,"[-0.13, 0.15]"
2,ICC3,Single fixed raters,0.011,1.022,172,172,0.445,"[-0.14, 0.16]"
3,ICC1k,Average raters absolute,-0.012,0.988,172,173,0.531,"[-0.37, 0.25]"
4,ICC2k,Average random raters,0.020,1.022,172,172,0.445,"[-0.31, 0.27]"
5,ICC3k,Average fixed raters,0.021,1.022,172,172,0.445,"[-0.32, 0.27]"


In [32]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='attitude').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.054,1.113,172,173,0.241,"[-0.1, 0.2]"
1,ICC2,Single random raters,0.051,1.108,172,172,0.251,"[-0.1, 0.2]"
2,ICC3,Single fixed raters,0.051,1.108,172,172,0.251,"[-0.1, 0.2]"
3,ICC1k,Average raters absolute,0.102,1.113,172,173,0.241,"[-0.21, 0.33]"
4,ICC2k,Average random raters,0.098,1.108,172,172,0.251,"[-0.22, 0.33]"
5,ICC3k,Average fixed raters,0.097,1.108,172,172,0.251,"[-0.22, 0.33]"


In [33]:
icc = pg.intraclass_corr(data=df, targets='ID', raters='Rater', ratings='sensitivity').round(3)
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,-0.073,0.864,172,173,0.830,"[-0.22, 0.08]"
1,ICC2,Single random raters,0.002,1.005,172,172,0.486,"[-0.12, 0.13]"
2,ICC3,Single fixed raters,0.003,1.005,172,172,0.486,"[-0.15, 0.15]"
3,ICC1k,Average raters absolute,-0.157,0.864,172,173,0.830,"[-0.56, 0.14]"
4,ICC2k,Average random raters,0.004,1.005,172,172,0.486,"[-0.28, 0.23]"
5,ICC3k,Average fixed raters,0.005,1.005,172,172,0.486,"[-0.34, 0.26]"
